<a href="https://colab.research.google.com/github/GauravAero/Script_Writer/blob/main/Script_Writer_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script Writer Using CrewAI


In this notebook, we demonstrate how to build and collaborating agentic AI system using **CrewAI**. The system integrates multiple agents, each with a specific role, to:
1. Gather current information from the internet about given topic.
2. Make a conversational script out of the information.

---
### Workflow Overview
1. Define tools to support the agents, such as web search capabilities.
2. Create two agents for data gathering and script creation.
3. Define tasks corresponding to these agents.
4. Set up a crew that executes the tasks sequentially.
5. Run the crew to achieve the desired outcome.

---
### Code Walkthrough
Below is the implementation.


### Step 1: Import Required Libraries and Define Tools

In [1]:
## initial setup for Colab.
## if you are using this in local jupyter notebook, skip this cell
!pip install crewai
!pip install crewai-tools
!pip install duckduckgo-search
!pip install langchain-community
!pip install litellm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.7/366.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.5 MB/s eta 0

In [2]:
from crewai import Agent, Task
from crewai.tools import tool
from crewai.llm import LLM
from langchain_community.tools import DuckDuckGoSearchRun
from datetime import datetime
import os

# Current date for context
Now = datetime.now()
Today = Now.strftime("%d-%b-%Y")

# Define a web search tool
@tool("DuckDuckGo Search")
def search_tool(search_query: str):
    """Search the internet for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)


### Step 2: Define the Agents

In [3]:
from google.colab import userdata

# Retrieve your API key from Colab secrets
os.environ["GOOGLE_API_KEY"] = userdata.get('Gemini_key')

# Use CrewAI's built-in LLM configuration (Recommended)
from crewai import LLM

# Configure Gemini LLM directly through CrewAI
gemini_llm = LLM(
    model="gemini/gemini-2.5-flash",  # Use gemini/ prefix for CrewAI
    api_key=os.environ["GOOGLE_API_KEY"]
)

In [5]:
from crewai import Agent
#from langchain_community.tools.exa.tool import ExaSearchTool

## Agent designed to gather information about a topic from internet.
content_explorer = Agent (
    role = 'content explorer',
    goal = 'Gather and provide latest information about the topic from internet',
    llm = gemini_llm,
    # llm = 'groq/gemma2-9b-it',
    # llm = 'openrouter/deepseek/deepseek-chat-v3-0324:free',
    # llm = 'gpt-4o',
    verbose = True,
    backstory = ('You are an expert researcher, who can gather detailed information about a topic.\
                  Gather at least 10 information. Consider you are on : '+Today),
    #tools = [Exa_Search_Tool],
    cache = True,
    max_iter = 5
)

## Agent to create a script out of information.
## Back story sets the context for the agent.
# script_llm = LLM (model='groq/llama3-70b-8192', temperature=1.2)

# script_llm = LLM (model='gemini-2.5-flash', temperature=0)
Script_Writer = Agent (
    role = 'Script Writer',
    goal = 'With the details given to you create an interesting conversational script out of it',
    #llm = 'openrouter/deepseek/deepseek-chat-v3-0324:free',
    llm = gemini_llm,
    verbose = True,
    backstory = ('You are an expert in Shakesperean literature. You are very good in creating conversations with the given chain of information.\
                 Tell as a script in 200 words.\
                 Consider you are on '+Today),
)

os.environ['OTEL_SDK_DISABLED'] = "True"


### Step 3: Define the Tasks

In [6]:
from crewai import Task

# Task to gather Latest information
get_details = Task (
                      description = "Get latest, trending, interesting information and news about {topic}",
                      expected_output = "Latest news, interesting information and trivia about {topic}",
                      agent = content_explorer
                    )

## Task to create script.
create_a_script = Task (
                      description = "Considering the given details in time order make an interesting conversation",
                      expected_output = "A humorous conversation connecting key details",
                      agent = Script_Writer,
                      context = [get_details],
                      output_file = 'Script.txt'
                    )



### Step 4: Set Up the Crew

In [7]:
from crewai import Crew, Process
from datetime import datetime

# Callback function to print a timestamp
def timestamp(Input):
    print(datetime.now())

# Define the crew with agents and tasks in sequential process
crew = Crew (
    agents = [content_explorer, Script_Writer],
    tasks = [get_details, create_a_script],
    verbose = True,
    Process = Process.sequential,
    step_callback = timestamp
)

### Step 5: Run the Crew and Observe Results

In [8]:
# Run the crew with a specific stock
result = crew.kickoff (inputs={'topic' : "Iran Isral conflict"})


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 23405f22-5f78-4393-8751-3bcbef6d7eea                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: content explorer                                                                                        │
│                                                                                                                 │
│  Task: Get latest, trending, interesting information and news about Iran Isral conflict                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

2025-08-03 05:22:35.584284

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: content explorer                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Latest, Trending, and Interesting Information and News Regarding the Iran-Israel Conflict (as of             │
│  03-Aug-2025):**                                                                                                │
│                                                                                                                 │
│  1.  **Heightened Regional Tensions After Reported Israeli Strikes in Syria:** Reports from early August 2025   │
│  indicate a significant escalation following alleged Israeli airstrikes deep within Syrian territory,           │
│  targeting Iranian Revolutionary Guard Corps (IRGC) assets and advanced weapons convoys. These strikes, while   │
│  unconfirmed by Israel, have been widely reported by regional media as the most extensive in months, prompting  │
│  strong condemnation from Tehran and Damascus, and raising fears of immediate retaliation from Iranian-backed   │
│  militias.                                                                                                      │
│                                                                                                                 │
│  2.  **Iran's Nuclear Program Continues to Advance Amid Stalled Diplomacy:** The International Atomic Energy    │
│  Agency (IAEA) released its latest confidential report in late July 2025, revealing Iran's continued            │
│  enrichment of uranium to near-weapons-grade purity (around 60% or higher), and significant advancements in     │
│  its centrifuge cascades. Diplomatic efforts to revive the Joint Comprehensive Plan of Action (JCPOA) remain    │
│  largely stalled, with both sides blaming the other for a lack of commitment, amplifying international          │
│  concerns about a potential nuclear breakout.                                                                   │
│                                                                                                                 │
│  3.  **Hezbollah and IDF Maintain High Alert on Northern Border:** The Israel-Lebanon border remains a          │
│  flashpoint. Following the Syrian strikes, Hezbollah leadership issued strong warnings of a proportional        │
│  response, and the Israel Defense Forces (IDF) have significantly reinforced their northern command. Unmanned   │
│  aerial vehicle (UAV) incursions have reportedly increased from both sides, though major direct confrontations  │
│  have, for now, been avoided, largely due to ongoing international mediation efforts.                           │
│                                                                                                                 │
│  4.  **Red Sea Shipping Under Persistent Houthi Threat:** Houthi rebels in Yemen, widely seen as an Iranian     │
│  proxy, continue to pose a significant threat to international shipping in the Red Sea and Gulf of Aden.        │
│  Despite ongoing international naval patrols, several commercial vessels reported drone or missile encounters   │
│  in late July 2025, leading to continued rerouting of traffic around Africa and sustaining elevated global      │
│  shipping costs. This maritime domain has become a crucial battleground in the broader Iran-Israel proxy        │
│  conflict.                                                                                                      │
│                                                        

...

Output()

2025-08-03 05:22:44.618369

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Script Writer                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  [SCENE START]                                                                                                  │
│                                                                                                                 │
│  **INT. ROYAL ANTECHAMBER - DAY**                                                                               │
│                                                                                                                 │
│  *LORD ALARIC, a portly nobleman, paces fretfully. LADY SERAPHINA, fanning herself, looks on with a wry smile.  │
│  MASTER ELIAS, a scholar, consults a rolled parchment.*                                                         │
│                                                                                                                 │
│  **LORD ALARIC:**                                                                                               │
│  Hark, good Seraphina, and thou, Elias! The very air doth thicken with ill tidings! Syria, they say, doth       │
│  groan beneath Israeli blows, deep within her bosom. 'Tis the most extensive strife in months!                  │
│                                                                                                                 │
│  **LADY SERAPHINA:**                                                                                            │
│  Aye, my Lord, and Iran, forsooth, doth spin its atoms with a zeal most vexing, whilst diplomacy, a withered    │
│  vine, yields no fruit. They blame each other, like children caught with hands in the sweets!                   │
│                                                                                                                 │
│  **MASTER ELIAS:**                                                                                              │
│  And the northern border, alas, hums with unseen wings! Hezbollah vows such vengeance, whilst the IDF doth      │
│  brace for multi-fronted strife. Even the Red Sea feels the sting of Houthi's drone, making passage dear!       │
│                                                                                                                 │
│  **LORD ALARIC:**                                                                                               │
│  Yet Iran, though shackled by our sanctions, finds coin to fund its proxies! A financial wizardry, methinks,    │
│  or simply a deep, dark purse!                                                                                  │
│                                                                                                                 │
│  **LADY SERAPHINA:**                                                                                            │
│  And the cyber-fiends, they say, doth prick our very power grids! One almost hears the hum of thwarted          │
│  breaches, whilst their own banks suffer digital fits.                                                          │
│                                                                                                                 │
│  **MASTER ELIAS:**                                                                                              │
│  The world calls for peace, Lord Alaric, but their hardline stance, 'tis whispered, serves elections more than  │
│  nations. Ah, this "shadow war" with new-fangled drones

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 29a84b1f-f1e1-4eae-a376-b0dd7f6b2f66                                                                     │
│  Agent: Script Writer                                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 23405f22-5f78-4393-8751-3bcbef6d7eea                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: [SCENE START]                                                                                    │
│                                                                                                                 │
│  **INT. ROYAL ANTECHAMBER - DAY**                                                                               │
│                                                                                                                 │
│  *LORD ALARIC, a portly nobleman, paces fretfully. LADY SERAPHINA, fanning herself, looks on with a wry smile.  │
│  MASTER ELIAS, a scholar, consults a rolled parchment.*                                                         │
│                                                                                                                 │
│  **LORD ALARIC:**                                                                                               │
│  Hark, good Seraphina, and thou, Elias! The very air doth thicken with ill tidings! Syria, they say, doth       │
│  groan beneath Israeli blows, deep within her bosom. 'Tis the most extensive strife in months!                  │
│                                                                                                                 │
│  **LADY SERAPHINA:**                                                                                            │
│  Aye, my Lord, and Iran, forsooth, doth spin its atoms with a zeal most vexing, whilst diplomacy, a withered    │
│  vine, yields no fruit. They blame each other, like children caught with hands in the sweets!                   │
│                                                                                                                 │
│  **MASTER ELIAS:**                                                                                              │
│  And the northern border, alas, hums with unseen wings! Hezbollah vows such vengeance, whilst the IDF doth      │
│  brace for multi-fronted strife. Even the Red Sea feels the sting of Houthi's drone, making passage dear!       │
│                                                                                                                 │
│  **LORD ALARIC:**                                                                                               │
│  Yet Iran, though shackled by our sanctions, finds coin to fund its proxies! A financial wizardry, methinks,    │
│  or simply a deep, dark purse!                                                                                  │
│                                                                                                                 │
│  **LADY SERAPHINA:**                                                                                            │
│  And the cyber-fiends, they say, doth prick our very power grids! One almost hears the hum of thwarted          │
│  breaches, whilst their own banks suffer digital fits.                                                          │
│                                                                                                                 │
│  **MASTER ELIAS:**                                                                                              │
│  The world calls for peace, Lord Alaric, but their har


---
### Conclusion
This workflow demonstrates how to integrate multiple agents to perform tasks sequentially in a context. It showcases the power of CrewAI for building AI-driven collaborative solutions.
